In [1]:
%load_ext autoreload
%autoreload 2


In [3]:
from spark.spark_config import create_spark_session
spark = create_spark_session()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-80880702-18a3-4684-9dc0-e393f16d995a;1.0
	confs: [default]
	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.0 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in central
	found com.typesafe#config;1.4.1 in central
	found org.slf4j#slf4j-api;1.7.26 in central
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in central
	found com.gith

In [4]:
from data_loader import DataLoader
from preprocessor import Preprocessor
from nlp_processor import NLPProcessor
from feature_extractor import FeatureExtractor


In [5]:
data_loader = DataLoader(spark)
df = data_loader.load_sample()

In [6]:
# analyser = EdaAnalyser(df)
# analyser.run_full_eda_report(['title', 'lyrics', 'views'])


In [7]:
df = Preprocessor.run(df)


In [8]:
df_tokenized = NLPProcessor.run(df)
df_tokenized.cache()

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [9]:
extractor = FeatureExtractor()
# extractor.fit(df_tokenized)
# df_final = extractor.transform(df_tokenized)

DataFrame[title: string, tag: string, artist: string, year: int, views: int, features: string, lyrics: string, id: int, language_cld3: string, language_ft: string, language: string, lyrics_cleaned: string, words_lemmatized: array<string>]

In [10]:
from pathlib import Path
base_dir = Path.cwd().parent
test_file_path = base_dir / "models" / "test" / "word2vec"
# extractor.save(str(test_file_path))
df_final = extractor.load(str(test_file_path)).transform(df_tokenized)

In [11]:
df_final.select("feature_array").show(1)

[Stage 7:>                                                          (0 + 1) / 1]

+--------------------+
|       feature_array|
+--------------------+
|[-0.0040902979754...|
+--------------------+
only showing top 1 row



In [12]:
from repository.cassandra_provider import CassandraProvider 
cassandra_provider = CassandraProvider()

In [13]:
cassandra_provider.save(df_final)